In [1]:
from Bio import SeqIO
import os,sys
import pandas as pd
import phylopandas as ph

In [7]:
## convert the dollo csv file to fasta
dollo=pd.read_csv("iVAX/RSVB_F_sum/RSVB_F_classII_dollo.csv")
## add meta information 
meta = pd.read_csv("Data_collection/HRSV_0622_meta_code_0810.csv")
fasta=pd.merge(dollo, meta, how='left', left_on='Accession', right_on='Accession_x')
fasta=fasta.filter(['Accession','dollo',"collection_date_guess","country_code","year","genotype"], axis=1)
## drop the sane pattern from same year and country
fasta=fasta.dropna(subset=['collection_date_guess','country_code'])
fasta["date"] = pd.to_datetime(fasta["collection_date_guess"])

fasta = fasta.sort_values(by="date")
fasta=fasta.drop_duplicates(subset=['dollo','year'])
fasta['taxa']=fasta.apply(lambda x:'%s|%s|%s|%s' % 
                          (x['Accession'],x['collection_date_guess'],x['country_code'],x['genotype']),axis=1)

fasta=fasta.filter(['taxa','dollo'], axis=1)
## subsample for the test run
sample = fasta.sample(frac=0.3)
sample
sample.to_csv("temp.tab", sep="\t",index = False,header=False)
fasta = SeqIO.parse("temp.tab", "tab")

SeqIO.write(fasta, "iVAX/RSVB_F_sum/RSVB_F_test.fasta", "fasta")
#os.remove("temp.tab")
## covert to a fasta format


/Users/jianichen1/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


137

In [3]:
def random_sample(df,n,file):
    sample = df.sample(frac=n)
    taxa = sample['taxa']
    taxa.to_csv(file, header = False, index =False)
    

In [4]:
##subset the dollo file 
def subsample(df,n,file):
    #Convert 'Date' to a datetime datatype.
    df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
    sample = df.groupby(['country',df['date'].dt.year],group_keys=False).apply(lambda x:x.sample(min(len(x),n)))
    taxa = sample['taxa']
    ## write taxa to a file
    taxa.to_csv(file, header = False, index =False)

In [23]:
## convert the dollo csv file to fasta
dollo=pd.read_csv("iVAX/RSVB_F_sum/RSVB_F_classII_Babel.csv")
## add meta information 
meta = pd.read_csv("Data_collection/HRSV_0622_meta_code_0810.csv")
fasta=pd.merge(dollo, meta, how='left', left_on='Accession', right_on='Accession_x')
fasta=fasta.filter(['Accession','Babel',"collection_date_guess","country_code","year","genotype"], axis=1)
fasta=fasta.dropna(subset=['collection_date_guess','country_code'])
fasta["date"] = pd.to_datetime(fasta["collection_date_guess"])

fasta = fasta.sort_values(by="date")
fasta=fasta.drop_duplicates(subset=['Babel'])
fasta['taxa']=fasta.apply(lambda x:'%s|%s|%s|%s' % 
                          (x['Accession'],x['collection_date_guess'],x['country_code'],x['genotype']),axis=1)
fasta=fasta.filter(['taxa','Babel'], axis=1)
fasta.to_csv("temp.tab", sep="\t",index = False,header=False)
fasta = SeqIO.parse("temp.tab", "tab")
fasta
SeqIO.write(fasta, "iVAX/RSVB_F_sum/HRSVB_F_epitope_Babel0818.fasta", "fasta")
os.remove("temp.tab")
## covert to a fasta format

In [9]:
## convert fasta to nexus
import dendropy

In [11]:
fasta = dendropy.DnaCharacterMatrix.get(path="iVAX/RSVB_F_sum/RSVB_F_test.fasta", schema="fasta")
fasta.write(path="iVAX/RSVB_F_sum/RSVB_F_test.nex", schema="nexus")

DataParseError: Error parsing data source 'iVAX/RSVB_F_sum/RSVB_F_test.fasta' on line 2 at column 1: Unrecognized sequence symbol '0'